In [1]:
import tensorflow as tf
import json
import numpy as np
import cv2

import matplotlib.pyplot as plt

from cell_division.nets.transfer_learning import CNN
from auxiliary.data.dataset_cell import CellDataset
from auxiliary import values as v
from auxiliary.utils.colors import bcolors as c

# from focal_loss import SparseCategoricalFocalLoss
from tensorflow.keras.losses import CategoricalCrossentropy
from cell_division.nets.custom_layers import w_cel_loss, focal_loss

from sklearn.utils.class_weight import compute_class_weight

# GPU config
from auxiliary.utils.timer import LoadingBar
from auxiliary.gpu.gpu_tf import (
    increase_gpu_memory, 
    set_gpu_allocator, 
    clear_session
)

increase_gpu_memory()
set_gpu_allocator()

In [2]:
img_dir = v.data_path + 'CellDivision/images/'
label_train_dir = v.data_path + 'CellDivision/train.csv'
label_test_dir = v.data_path + 'CellDivision/test.csv'
label_val_dir = v.data_path + 'CellDivision/val.csv'

INPUT_SHAPE = (50, 50, 3)
BATCH_SIZE = 64

Dataset (Generators)

Generatos do not load directly the images into memory, but they load the images on the fly. This is useful when the dataset is too large to fit into memory.

In [3]:
train_generator = CellDataset(
    img_dir, 
    label_train_dir, 
    batch_size=BATCH_SIZE, 
    resize=INPUT_SHAPE[:2]
)

val_generator = CellDataset(
    img_dir, 
    label_val_dir, 
    batch_size=BATCH_SIZE, 
    resize=INPUT_SHAPE[:2]
)

Transfer Learning 

In [4]:
base_models = {
    'DenseNet121': tf.keras.applications.DenseNet121,
    'EfficientNetV2L': tf.keras.applications.EfficientNetV2L,
    'EfficientNetV2M': tf.keras.applications.EfficientNetV2M,
    'VGG16': tf.keras.applications.VGG16,
    'ResNet50': tf.keras.applications.ResNet50,
    'InceptionV3': tf.keras.applications.InceptionV3,
    'MobileNetV2': tf.keras.applications.MobileNetV2,
    'NASNetMobile': tf.keras.applications.NASNetMobile,
}


In [5]:
param_grid = {
    'base_model': list(base_models.keys()),
    'lr': [1e-3, 1e-2],
    'fine_tune': [True, False],
    'loss': [focal_loss(), w_cel_loss()],
    'top': ['CAM', 'Standard'],
    # 'class_weight': [None, 'balanced']
}

In [6]:
from tensorboard.errors import InvalidArgumentError

bar = LoadingBar(
    len(param_grid['base_model']) * len(param_grid['lr']) * len(param_grid['fine_tune']) * len(param_grid['loss']) * len(param_grid['top'])
)

results = {}

for base_model in param_grid['base_model']:
    for lr in param_grid['lr']:
        for fine_tune in param_grid['fine_tune']:
            for loss in param_grid['loss']:
                for top in param_grid['top']:
                    print(f'{c.OKGREEN}Model: {base_model} - LR: {lr} - Fine Tune: {fine_tune} - Loss: {loss} - Top: {top}{c.ENDC}')
                    
                    try:
                        model = CNN(
                            base=base_models[base_model],
                            n_classes=3,
                            input_shape=INPUT_SHAPE,
                            fine_tune=fine_tune
                        )
                        model.build_top(activation='softmax', b_type=top)
                        model.compile(lr=lr, loss=loss)
                        model.fit(
                            train_generator,
                            val_generator,
                            epochs=100,
                            batch_size=BATCH_SIZE,
                            save=False,
                            verbose=1
                        )
    
                        results[
                            str((base_model, lr, fine_tune, loss.__name__, top))
                        ] = model.model.history.history
                    except Exception as e:
                        print(f'{c.FAIL}Error: {e}{c.ENDC}')
                        results[(base_model, lr, fine_tune, loss, top)] = None

                    clear_session()
                    bar.update()
                    
    with open(f'../cell_division/results/{base_model}.json', 'w') as f:
        json.dump(results, f)
    results = {}

bar.end()

Model: DenseNet121 - LR: 0.001 - Fine Tune: True - Loss: <function focal_loss.<locals>.focal_loss_fixed at 0x700d0b328790> - Top: CAM
Epoch 1/100
12/12 [==============================] - 35s 2s/step - loss: 0.3170 - auc: 0.6351 - val_loss: 2.4222 - val_auc: 0.6591 - lr: 0.0010
Epoch 2/100
12/12 [==============================] - 11s 918ms/step - loss: 0.0633 - auc: 0.7134 - val_loss: 2.4222 - val_auc: 0.6591 - lr: 0.0010
Epoch 3/100
12/12 [==============================] - 11s 893ms/step - loss: 0.0679 - auc: 0.8226 - val_loss: 4.0692 - val_auc: 0.4273 - lr: 0.0010
Epoch 4/100
12/12 [==============================] - 11s 916ms/step - loss: 0.0461 - auc: 0.8658 - val_loss: 4.0526 - val_auc: 0.4265 - lr: 0.0010
Epoch 5/100
12/12 [==============================] - 11s 967ms/step - loss: 0.0362 - auc: 0.9068 - val_loss: 2.3432 - val_auc: 0.6617 - lr: 0.0010
Epoch 6/100
12/12 [==============================] - 12s 1s/step - loss: 0.0286 - auc: 0.9434 - val_loss: 2.0856 - val_auc: 0.6701 - l

TypeError: Object of type float32 is not JSON serializable

In [ ]:
print(json.dumps(results, indent=4))
# Save
with open('../cell_division/results/grid_search_cnn.json', 'w') as f:
    json.dump(results, f)